# 🎬 유튜브 댓글 분석: YouTube API + KoBERT 감성분석 + 혐오표현 탐지

## 학습 목표
1. **YouTube Data API**를 활용하여 공식적으로 댓글을 수집한다
2. **KoBERT 기반 딥러닝 감성분석**을 수행한다
3. 혐오표현을 탐지하고 분류한다
4. 분석 결과를 시각화한다

---

## 📚 목차
1. [환경 설정](#1-환경-설정)
2. [YouTube API 설정](#2-youtube-api-설정)
3. [동영상 검색 및 댓글 수집](#3-동영상-검색-및-댓글-수집)
4. [텍스트 전처리](#4-텍스트-전처리)
5. [KoBERT 감성분석](#5-kobert-감성분석)
6. [혐오표현 탐지](#6-혐오표현-탐지)
7. [종합 분석 및 시각화](#7-종합-분석-및-시각화)

---

## 🔑 YouTube Data API란?
- Google에서 제공하는 공식 YouTube API
- 동영상 검색, 채널 정보, 댓글 수집 등 가능
- **일일 할당량**: 10,000 units (댓글 1회 요청 ≈ 1 unit)
- 무료로 사용 가능 (Google Cloud Console에서 API 키 발급)

---
# 1. 환경 설정

In [ ]:
# ============================================
# 1-1. 필수 라이브러리 설치
# ============================================

# YouTube API 클라이언트
!pip install google-api-python-client -q

# 한국어 자연어처리
!pip install konlpy -q

# 데이터 분석 및 시각화
!pip install pandas numpy matplotlib seaborn wordcloud -q

# 토픽 모델링
!pip install gensim -q

# KoBERT 딥러닝
!pip install transformers torch -q
!pip install sentencepiece -q

print("✅ 라이브러리 설치 완료!")

In [ ]:
# ============================================
# 1-2. Colab 환경 설정 (Java, 한글 폰트)
# ============================================

import os
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

def setup_colab_environment():
    """Colab 환경 설정: Java + 한글 폰트"""
    
    print("🔄 Java 설치 중...")
    os.system('apt-get update -qq')
    os.system('apt-get install -y openjdk-11-jdk-headless -qq')
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
    
    print("🔄 한글 폰트 설치 중...")
    os.system('apt-get install -y fonts-nanum -qq')
    fm._load_fontmanager()
    
    plt.rcParams['font.family'] = 'NanumBarunGothic'
    plt.rcParams['axes.unicode_minus'] = False
    
    print("✅ 환경 설정 완료!")

try:
    import google.colab
    setup_colab_environment()
    IN_COLAB = True
except ImportError:
    print("로컬 환경입니다.")
    plt.rcParams['font.family'] = 'Malgun Gothic'
    IN_COLAB = False

In [ ]:
# ============================================
# 1-3. 라이브러리 임포트
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
from collections import Counter
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# YouTube API
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# 한국어 NLP
from konlpy.tag import Okt

# 딥러닝
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

print("✅ 모든 라이브러리 임포트 완료!")
print(f"📅 실행 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 PyTorch: {torch.__version__}")
print(f"🔧 GPU 사용 가능: {torch.cuda.is_available()}")

---
# 2. YouTube API 설정

In [ ]:
# ============================================
# 2-1. API 키 및 검색 설정
# ============================================

# ⬇️⬇️⬇️ 여기를 수정하세요! ⬇️⬇️⬇️

# YouTube Data API 키
API_KEY = "AIzaSyBFsjYPVWNQml-4vLG9zeSV8mwVM(예시임)"

# 검색 키워드
KEYWORD = "이태원참사"

# 수집 설정
MAX_VIDEOS = 10          # 검색할 최대 동영상 수
MAX_COMMENTS = 100       # 동영상당 최대 댓글 수

# ⬆️⬆️⬆️ 여기를 수정하세요! ⬆️⬆️⬆️

print(f"🔑 API 키: {API_KEY[:20]}...")
print(f"🔍 검색 키워드: {KEYWORD}")
print(f"🎬 최대 동영상 수: {MAX_VIDEOS}개")
print(f"💬 동영상당 최대 댓글: {MAX_COMMENTS}개")

In [ ]:
# ============================================
# 2-2. YouTube API 클라이언트 생성
# ============================================

def create_youtube_client(api_key):
    """
    YouTube Data API v3 클라이언트 생성
    
    Parameters:
    -----------
    api_key : str
        Google API 키
    
    Returns:
    --------
    googleapiclient.discovery.Resource
    """
    try:
        youtube = build('youtube', 'v3', developerKey=api_key)
        print("✅ YouTube API 클라이언트 생성 완료!")
        return youtube
    except Exception as e:
        print(f"❌ API 클라이언트 생성 실패: {e}")
        return None

# 클라이언트 생성
youtube = create_youtube_client(API_KEY)

In [ ]:
# ============================================
# 2-3. API 연결 테스트
# ============================================

def test_api_connection(youtube):
    """API 연결 테스트"""
    try:
        request = youtube.search().list(
            part='snippet',
            q='test',
            type='video',
            maxResults=1
        )
        response = request.execute()
        print("✅ API 연결 테스트 성공!")
        return True
    except HttpError as e:
        print(f"❌ API 오류: {e}")
        return False
    except Exception as e:
        print(f"❌ 연결 실패: {e}")
        return False

# API 테스트
if youtube:
    api_works = test_api_connection(youtube)
else:
    api_works = False

---
# 3. 동영상 검색 및 댓글 수집

In [ ]:
# ============================================
# 3-1. 키워드로 동영상 검색
# ============================================

def search_videos(youtube, keyword, max_results=10):
    """
    키워드로 YouTube 동영상 검색
    
    Parameters:
    -----------
    youtube : googleapiclient.discovery.Resource
        YouTube API 클라이언트
    keyword : str
        검색 키워드
    max_results : int
        최대 검색 결과 수
    
    Returns:
    --------
    list : 동영상 정보 리스트
    """
    
    videos = []
    
    print(f"🔄 '{keyword}' 동영상 검색 중...")
    
    try:
        request = youtube.search().list(
            part='snippet',
            q=keyword,
            type='video',
            order='relevance',  # 관련성순 (viewCount: 조회수순, date: 최신순)
            regionCode='KR',
            relevanceLanguage='ko',
            maxResults=max_results
        )
        response = request.execute()
        
        for item in response.get('items', []):
            video_id = item['id']['videoId']
            snippet = item['snippet']
            
            videos.append({
                'video_id': video_id,
                'title': snippet['title'],
                'channel': snippet['channelTitle'],
                'published_at': snippet['publishedAt'][:10],
                'url': f'https://www.youtube.com/watch?v={video_id}'
            })
        
        print(f"✅ {len(videos)}개 동영상 검색 완료!")
        
    except HttpError as e:
        print(f"❌ API 오류: {e}")
    except Exception as e:
        print(f"❌ 검색 실패: {e}")
    
    return videos

# 동영상 검색
if youtube and api_works:
    videos = search_videos(youtube, KEYWORD, MAX_VIDEOS)
else:
    videos = []

In [ ]:
# ============================================
# 3-2. 검색된 동영상 목록 확인
# ============================================

if videos:
    print(f"\n📋 검색된 동영상 목록 ({len(videos)}개):")
    print("="*70)
    
    for i, video in enumerate(videos, 1):
        title = video['title'][:50] + "..." if len(video['title']) > 50 else video['title']
        print(f"\n{i}. {title}")
        print(f"   📺 채널: {video['channel']}")
        print(f"   📅 게시일: {video['published_at']}")
        print(f"   🔗 {video['url']}")
else:
    print("❌ 검색된 동영상이 없습니다.")

In [ ]:
# ============================================
# 3-3. 동영상 댓글 수집 함수
# ============================================

def get_video_comments(youtube, video_id, max_comments=100):
    """
    동영상의 댓글 수집 (YouTube API)
    
    Parameters:
    -----------
    youtube : googleapiclient.discovery.Resource
        YouTube API 클라이언트
    video_id : str
        동영상 ID
    max_comments : int
        최대 댓글 수
    
    Returns:
    --------
    list : 댓글 데이터 리스트
    """
    
    comments = []
    next_page_token = None
    
    try:
        while len(comments) < max_comments:
            request = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                order='relevance',  # 관련성순 (time: 최신순)
                maxResults=min(100, max_comments - len(comments)),
                pageToken=next_page_token,
                textFormat='plainText'
            )
            response = request.execute()
            
            for item in response.get('items', []):
                comment = item['snippet']['topLevelComment']['snippet']
                
                comments.append({
                    'text': comment['textDisplay'],
                    'author': comment['authorDisplayName'],
                    'likes': comment['likeCount'],
                    'published_at': comment['publishedAt'][:10],
                    'reply_count': item['snippet']['totalReplyCount']
                })
            
            # 다음 페이지
            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break
        
    except HttpError as e:
        # 댓글이 비활성화된 동영상
        if 'commentsDisabled' in str(e):
            pass
        else:
            pass  # 다른 오류 무시
    except Exception as e:
        pass
    
    return comments

In [ ]:
# ============================================
# 3-4. 모든 동영상에서 댓글 수집
# ============================================

all_comments = []

if videos and youtube:
    print(f"🔄 {len(videos)}개 동영상에서 댓글 수집 시작...\n")
    
    for i, video in enumerate(videos, 1):
        title_short = video['title'][:40] + "..." if len(video['title']) > 40 else video['title']
        print(f"[{i}/{len(videos)}] {title_short}")
        
        comments = get_video_comments(youtube, video['video_id'], MAX_COMMENTS)
        
        # 동영상 정보 추가
        for comment in comments:
            comment['video_id'] = video['video_id']
            comment['video_title'] = video['title']
            comment['channel'] = video['channel']
        
        all_comments.extend(comments)
        print(f"      ✅ {len(comments)}개 댓글 수집")
    
    print(f"\n{'='*50}")
    print(f"✅ 총 {len(all_comments)}개 댓글 수집 완료!")
else:
    print("⚠️ 동영상이 없거나 API 연결 실패")

In [ ]:
# ============================================
# 3-5. 데이터프레임 생성 (또는 샘플 데이터)
# ============================================

def create_sample_comments():
    """샘플 댓글 데이터 생성"""
    sample_data = [
        # 애도/추모
        {"text": "유가족분들께 깊은 위로를 전합니다. 다시는 이런 일이 없어야 합니다.", "likes": 1245},
        {"text": "희생자분들의 명복을 빕니다. 진상규명이 반드시 이루어져야 합니다.", "likes": 987},
        {"text": "너무 안타깝습니다. 삼가 고인의 명복을 빕니다.", "likes": 876},
        {"text": "잊지 않겠습니다. 희생자분들을 기억합니다.", "likes": 543},
        {"text": "유가족분들 힘내세요. 응원합니다.", "likes": 432},
        
        # 비판/분노
        {"text": "책임자 처벌이 제대로 이뤄져야 합니다. 이게 나라입니까?", "likes": 2134},
        {"text": "경찰은 뭘 했나요? 그 많은 신고를 무시하다니 말이 됩니까?", "likes": 1876},
        {"text": "정부의 무능이 만든 참사입니다. 책임지는 사람이 없네요.", "likes": 1432},
        {"text": "아직도 책임지는 사람이 없다니 화가 납니다.", "likes": 1234},
        {"text": "너무 분노가 치밀어 오릅니다. 왜 이런 일이 반복되나요?", "likes": 1123},
        {"text": "안전 불감증의 결과입니다. 정말 답답합니다.", "likes": 987},
        
        # 정치 공방
        {"text": "또 정치적으로 이용하려고? 희생자들 두 번 죽이지 마세요.", "likes": 1567},
        {"text": "야당은 맨날 정쟁만... 진짜 추모할 생각이나 있는 건가?", "likes": 1345},
        {"text": "여당이나 야당이나 정치적 이용만 하려고 하네요.", "likes": 1123},
        
        # 혐오/갈등 표현
        {"text": "그쪽 사람들은 항상 이런 식으로 선동하더라.", "likes": 456},
        {"text": "586들이 문제야. 맨날 자기들 기준으로만 판단함.", "likes": 345},
        {"text": "좌파들 특징: 뭐든 정치화시킴", "likes": 287},
        {"text": "수꼴들은 정부 비판하면 다 종북으로 모나", "likes": 234},
        {"text": "요즘 젊은것들은 왜 그러냐", "likes": 198},
        {"text": "틀딱들이 투표를 잘못해서 이 지경이야", "likes": 176},
        
        # 피해자 비난
        {"text": "왜 거기를 갔어? 위험한 줄 알면서...", "likes": 123},
        {"text": "할로윈에 그렇게 많이 모이면 그렇게 되지", "likes": 98},
        
        # 중립적
        {"text": "정확한 사실관계 확인이 먼저입니다.", "likes": 567},
        {"text": "재발 방지 대책이 중요합니다.", "likes": 489},
        {"text": "양쪽 다 진정하고 사실에 기반해서 논의합시다.", "likes": 345},
        {"text": "추모와 진상규명 둘 다 필요합니다.", "likes": 298},
        {"text": "객관적인 조사가 필요한 시점입니다.", "likes": 234},
        {"text": "감정적 대응보다 냉정한 분석이 필요해요.", "likes": 198},
    ]
    
    df = pd.DataFrame(sample_data)
    df['video_id'] = 'sample_video'
    df['video_title'] = f'{KEYWORD} 관련 영상'
    df['channel'] = np.random.choice(['JTBC News', 'MBC News', 'KBS News', 'SBS News'], len(df))
    df['author'] = [f'user_{i}' for i in range(len(df))]
    df['published_at'] = '2024-10-29'
    df['reply_count'] = np.random.randint(0, 50, len(df))
    
    print(f"✅ 샘플 데이터 생성 완료! ({len(df)}개 댓글)")
    return df

# 데이터프레임 생성
if all_comments:
    df_comments = pd.DataFrame(all_comments)
    print(f"📊 수집된 데이터: {len(df_comments)}개 댓글")
else:
    print("⚠️ 수집된 댓글이 없어 샘플 데이터를 사용합니다.")
    df_comments = create_sample_comments()

df_comments.head()

In [ ]:
# ============================================
# 3-6. 데이터 기본 정보 확인
# ============================================

print("="*50)
print("📊 데이터 기본 정보")
print("="*50)
print(f"총 댓글 수: {len(df_comments):,}개")
print(f"동영상 수: {df_comments['video_id'].nunique()}개")
print(f"평균 좋아요: {df_comments['likes'].mean():.1f}개")
print(f"최대 좋아요: {df_comments['likes'].max():,}개")

print("\n📺 채널별 댓글 수:")
print(df_comments['channel'].value_counts().head())

---
# 4. 텍스트 전처리

In [ ]:
# ============================================
# 4-1. 형태소 분석기 및 전처리 함수
# ============================================

okt = Okt()

STOPWORDS = {
    '것', '수', '등', '때', '중', '좀', '더', '네', '거', '뭐', '제', '저', '게',
    '이', '그', '저', '여기', '거기', '나', '너', '우리', '저희',
    '하다', '되다', '있다', '없다', '같다', '보다', '주다',
    '안', '못', '잘', '왜', '어떻게', '언제', '어디', '누구', '무엇',
    '진짜', '정말', '너무', '매우', '아주', '완전', '약간', '조금',
    '그냥', '바로', '다시', '또', '이미', '아직', '벌써', '곧',
    '근데', '그런데', '하지만', '그래서', '그러나', '따라서',
    '영상', '댓글', '구독', '좋아요', '알림', '채널',
    '분', '초', '편', '화', '회', '번', '개', '명', '년', '월', '일',
}

def clean_text(text):
    """텍스트 정규화"""
    if pd.isna(text):
        return ""
    text = str(text)
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def extract_keywords(text, min_length=2):
    """명사 키워드 추출"""
    if not text:
        return []
    nouns = okt.nouns(str(text))
    return [w for w in nouns if len(w) >= min_length and w not in STOPWORDS]

print("✅ 전처리 함수 정의 완료!")

In [ ]:
# ============================================
# 4-2. 전처리 적용
# ============================================

print("🔄 전처리 진행 중...")

df_comments['cleaned_text'] = df_comments['text'].apply(clean_text)
df_comments['keywords'] = df_comments['cleaned_text'].apply(extract_keywords)
df_comments['text_length'] = df_comments['cleaned_text'].apply(len)

# 빈 텍스트 제거
df_comments = df_comments[df_comments['text_length'] > 0].reset_index(drop=True)

print(f"✅ 전처리 완료! (유효 댓글: {len(df_comments)}개)")

In [ ]:
# ============================================
# 4-3. 워드클라우드
# ============================================

all_keywords = []
for keywords in df_comments['keywords']:
    all_keywords.extend(keywords)

word_freq = Counter(all_keywords)

if word_freq:
    if IN_COLAB:
        font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
    else:
        font_path = 'C:/Windows/Fonts/malgun.ttf'
    
    wordcloud = WordCloud(
        font_path=font_path,
        width=900, height=400,
        background_color='white',
        colormap='RdYlBu_r',
        max_words=80
    ).generate_from_frequencies(word_freq)

    plt.figure(figsize=(14, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'📊 "{KEYWORD}" 유튜브 댓글 키워드', fontsize=16, fontweight='bold')
    plt.show()

# TOP 20 키워드
print("\n📊 TOP 20 키워드:")
for word, count in word_freq.most_common(20):
    bar = '█' * min(count // 2, 20)
    print(f"{word:10} | {count:3}회 | {bar}")

---
# 5. KoBERT 감성분석

In [ ]:
# ============================================
# 5-1. KoBERT 기반 감성분석 모델 로드
# ============================================

print("🔄 KoBERT 기반 감성분석 모델 로딩 중...")
print("   (최초 실행 시 모델 다운로드에 시간이 걸릴 수 있습니다)")

# 사용할 모델 선택
MODEL_NAME = "snunlp/KR-FinBert-SC"  # 3클래스 (positive, negative, neutral)

# 대안 모델들:
# MODEL_NAME = "jason9693/soongsil-bert-base-nsmc"  # 2클래스
# MODEL_NAME = "beomi/KcELECTRA-base"  # 한국어 댓글 특화

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
    
    sentiment_analyzer = pipeline(
        "sentiment-analysis",
        model=model,
        tokenizer=tokenizer,
        device=0 if torch.cuda.is_available() else -1,
        max_length=512,
        truncation=True
    )
    
    print(f"\n✅ 모델 로드 완료!")
    print(f"   모델: {MODEL_NAME}")
    print(f"   디바이스: {'GPU' if torch.cuda.is_available() else 'CPU'}")
    MODEL_LOADED = True
    
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    MODEL_LOADED = False

In [ ]:
# ============================================
# 5-2. 감성분석 함수 정의
# ============================================

def analyze_sentiment_kobert(text):
    """
    KoBERT 기반 감성분석
    
    Returns: (라벨, 신뢰도, 정규화 점수)
    """
    if not text or pd.isna(text) or len(str(text).strip()) < 2:
        return 'neutral', 0.5, 0.0
    
    try:
        text = str(text)[:500]
        result = sentiment_analyzer(text)[0]
        
        label = result['label'].lower()
        score = result['score']
        
        # 라벨 정규화
        if 'positive' in label or 'pos' in label or label == '1':
            return 'positive', score, score
        elif 'negative' in label or 'neg' in label or label == '0':
            return 'negative', score, -score
        else:
            return 'neutral', score, 0.0
            
    except:
        return 'neutral', 0.5, 0.0

# 규칙 기반 백업
POSITIVE_WORDS = {'좋다', '좋아', '감사', '응원', '사랑', '희망', '위로', '명복', '추모', '힘내'}
NEGATIVE_WORDS = {'싫다', '화나다', '분노', '실망', '답답', '문제', '잘못', '무능', '비판', '처벌'}

def analyze_sentiment_rule(text):
    """규칙 기반 감성분석 (백업용)"""
    if not text:
        return 'neutral', 0.5, 0.0
    text = str(text).lower()
    morphs = okt.morphs(text)
    pos = sum(1 for w in morphs if w in POSITIVE_WORDS)
    neg = sum(1 for w in morphs if w in NEGATIVE_WORDS)
    if pos > neg:
        return 'positive', 0.7, pos - neg
    elif neg > pos:
        return 'negative', 0.7, -(neg - pos)
    return 'neutral', 0.5, 0.0

print("✅ 감성분석 함수 정의 완료!")

In [ ]:
# ============================================
# 5-3. KoBERT 감성분석 테스트
# ============================================

if MODEL_LOADED:
    test_texts = [
        "유가족분들께 깊은 위로를 전합니다.",
        "정말 화가 납니다. 책임자 처벌해야 합니다.",
        "객관적인 조사가 필요합니다.",
        "그쪽 사람들은 항상 이런 식이야."
    ]
    
    print("🧪 KoBERT 감성분석 테스트:")
    print("="*60)
    for text in test_texts:
        label, conf, score = analyze_sentiment_kobert(text)
        print(f"\n'{text[:40]}...'")
        print(f"  → {label} (신뢰도: {conf:.3f})")

In [ ]:
# ============================================
# 5-4. 전체 데이터에 감성분석 적용
# ============================================

print("🔄 KoBERT 감성분석 진행 중...")
print(f"   총 {len(df_comments)}개 댓글 분석")

sentiments = []
confidences = []
scores = []

for i, text in enumerate(df_comments['cleaned_text']):
    if MODEL_LOADED:
        label, conf, score = analyze_sentiment_kobert(text)
    else:
        label, conf, score = analyze_sentiment_rule(text)
    
    sentiments.append(label)
    confidences.append(conf)
    scores.append(score)
    
    if (i + 1) % 50 == 0:
        print(f"   진행: {i+1}/{len(df_comments)}")

df_comments['sentiment'] = sentiments
df_comments['sentiment_confidence'] = confidences
df_comments['sentiment_score'] = scores

print("\n✅ 감성분석 완료!")
print("\n📊 감성 분포:")
print(df_comments['sentiment'].value_counts())

In [ ]:
# ============================================
# 5-5. 감성분석 시각화
# ============================================

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. 감성 분포
sentiment_counts = df_comments['sentiment'].value_counts()
colors = {'positive': '#27ae60', 'neutral': '#95a5a6', 'negative': '#e74c3c'}
pie_colors = [colors.get(s, '#95a5a6') for s in sentiment_counts.index]

axes[0].pie(sentiment_counts.values, 
            labels=['긍정', '중립', '부정'][:len(sentiment_counts)],
            autopct='%1.1f%%', colors=pie_colors, startangle=90)
axes[0].set_title('🤖 KoBERT 감성 분포', fontsize=12, fontweight='bold')

# 2. 신뢰도 분포
axes[1].hist(df_comments['sentiment_confidence'], bins=20, color='steelblue',
             edgecolor='white', alpha=0.7)
axes[1].axvline(df_comments['sentiment_confidence'].mean(), color='red', linestyle='--',
                label=f'평균: {df_comments["sentiment_confidence"].mean():.2f}')
axes[1].set_xlabel('신뢰도')
axes[1].set_ylabel('빈도')
axes[1].set_title('📊 신뢰도 분포', fontsize=12, fontweight='bold')
axes[1].legend()

# 3. 감성별 평균 좋아요
sentiment_likes = df_comments.groupby('sentiment')['likes'].mean().sort_values()
bar_colors = [colors.get(s, '#95a5a6') for s in sentiment_likes.index]
axes[2].barh(sentiment_likes.index, sentiment_likes.values, color=bar_colors)
axes[2].set_xlabel('평균 좋아요')
axes[2].set_title('📈 감성별 평균 좋아요', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

---
# 6. 혐오표현 탐지

In [ ]:
# ============================================
# 6-1. 혐오표현 패턴 정의
# ============================================

HATE_PATTERNS = {
    '일반화': [
        '항상', '맨날', '늘', '전부', '다', '걔네', '쟤네', '그쪽', '저쪽',
        '니들', '무조건', '어차피', '역시', '그놈', '그년'
    ],
    '비하': [
        '한심', '멍청', '무식', '바보', '병신', '또라이', '미친',
        '수준', '저질', '쓰레기', '찌질', '꼴통', '돼지', '벌레'
    ],
    '선동': [
        '각성', '일어나', '심판', '뽑아야', '없애야', '망해라', '척결',
        '죽여', '때려', '박멸', '청산', '응징'
    ],
    '정치혐오': [
        '좌파', '우파', '빨갱이', '수꼴', '꼴통', '진보충', '보수꼴',
        '문빠', '윤빠', '국힘', '민주당충', '찍충'
    ],
    '세대혐오': [
        '틀딱', '꼰대', '586', '젊은것', '늙은이', '요즘것들', 'MZ충',
        '라떼', '노인네', '애들', '어린것'
    ],
    '성별혐오': [
        '페미', '한남', '한녀', '남혐', '여혐', '김치녀', '맘충', '냄저'
    ],
    '피해자비난': [
        '왜 갔', '왜 거기', '자업자득', '니 탓', '니가 알아서',
        '그러니까', '조심했어야', '안 갔으면'
    ]
}

def detect_hate_speech(text):
    """혐오표현 탐지"""
    if not text or pd.isna(text):
        return [], [], 0
    
    text = str(text).lower()
    found_types = []
    found_words = []
    
    for hate_type, patterns in HATE_PATTERNS.items():
        for pattern in patterns:
            if pattern.lower() in text:
                if hate_type not in found_types:
                    found_types.append(hate_type)
                found_words.append(pattern)
    
    hate_score = len(found_types) * 2 + len(found_words)
    return found_types, found_words, hate_score

print("✅ 혐오표현 패턴 정의 완료!")

In [ ]:
# ============================================
# 6-2. 혐오표현 탐지 적용
# ============================================

print("🔄 혐오표현 탐지 중...")

hate_results = df_comments['cleaned_text'].apply(detect_hate_speech)

df_comments['hate_types'] = [r[0] for r in hate_results]
df_comments['hate_words'] = [r[1] for r in hate_results]
df_comments['hate_score'] = [r[2] for r in hate_results]
df_comments['has_hate'] = df_comments['hate_types'].apply(lambda x: len(x) > 0)

hate_count = df_comments['has_hate'].sum()
hate_ratio = hate_count / len(df_comments) * 100

print("✅ 완료!")
print(f"\n📊 혐오표현 탐지 결과:")
print(f"   전체: {len(df_comments)}개")
print(f"   혐오 포함: {hate_count}개 ({hate_ratio:.1f}%)")

In [ ]:
# ============================================
# 6-3. 혐오표현 유형별 분석
# ============================================

all_hate_types = []
for types in df_comments['hate_types']:
    all_hate_types.extend(types)

type_freq = Counter(all_hate_types)

print("\n📊 혐오표현 유형별 빈도:")
print("="*50)
for hate_type, count in type_freq.most_common():
    pct = count / len(df_comments) * 100
    bar = '█' * min(count, 20)
    print(f"{hate_type:12} | {count:3}개 ({pct:5.1f}%) | {bar}")

In [ ]:
# ============================================
# 6-4. 혐오표현 시각화
# ============================================

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. 혐오 비율
hate_data = [df_comments['has_hate'].sum(), (~df_comments['has_hate']).sum()]
axes[0].pie(hate_data, labels=['혐오 포함', '정상'], autopct='%1.1f%%',
            colors=['#e74c3c', '#3498db'], startangle=90, explode=[0.05, 0])
axes[0].set_title(f'⚠️ 혐오표현 비율 ({hate_ratio:.1f}%)', fontsize=12, fontweight='bold')

# 2. 유형별 분포
if type_freq:
    types, counts = zip(*type_freq.most_common())
    colors_hate = plt.cm.Reds(np.linspace(0.3, 0.9, len(types)))
    axes[1].barh(types, counts, color=colors_hate)
    axes[1].set_xlabel('빈도')
    axes[1].invert_yaxis()
axes[1].set_title('🚫 혐오표현 유형', fontsize=12, fontweight='bold')

# 3. 감성 vs 혐오
cross_tab = pd.crosstab(df_comments['sentiment'], df_comments['has_hate'])
cross_tab.columns = ['정상', '혐오']
cross_tab.plot(kind='bar', ax=axes[2], color=['#3498db', '#e74c3c'], edgecolor='white')
axes[2].set_xlabel('감성')
axes[2].set_ylabel('댓글 수')
axes[2].set_title('📊 감성별 혐오표현', fontsize=12, fontweight='bold')
axes[2].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

---
# 7. 종합 분석 및 시각화

In [ ]:
# ============================================
# 7-1. 종합 대시보드
# ============================================

fig = plt.figure(figsize=(18, 12))

# 1. KoBERT 감성 분포
ax1 = fig.add_subplot(2, 3, 1)
sentiment_counts = df_comments['sentiment'].value_counts()
colors_sent = {'positive': '#27ae60', 'neutral': '#95a5a6', 'negative': '#e74c3c'}
pie_colors = [colors_sent.get(s, '#95a5a6') for s in sentiment_counts.index]
ax1.pie(sentiment_counts.values, labels=['긍정', '중립', '부정'][:len(sentiment_counts)],
        autopct='%1.1f%%', colors=pie_colors, startangle=90)
ax1.set_title('🤖 KoBERT 감성 분포', fontsize=12, fontweight='bold')

# 2. 혐오표현 비율
ax2 = fig.add_subplot(2, 3, 2)
hate_data = [df_comments['has_hate'].sum(), (~df_comments['has_hate']).sum()]
ax2.pie(hate_data, labels=['혐오 포함', '정상'], autopct='%1.1f%%',
        colors=['#e74c3c', '#3498db'], startangle=90, explode=[0.05, 0])
ax2.set_title(f'⚠️ 혐오표현 비율 ({hate_ratio:.1f}%)', fontsize=12, fontweight='bold')

# 3. 채널별 댓글 수
ax3 = fig.add_subplot(2, 3, 3)
channel_counts = df_comments['channel'].value_counts().head(7)
ax3.barh(channel_counts.index, channel_counts.values, color='steelblue')
ax3.set_xlabel('댓글 수')
ax3.invert_yaxis()
ax3.set_title('📺 채널별 댓글 수', fontsize=12, fontweight='bold')

# 4. 키워드 TOP 10
ax4 = fig.add_subplot(2, 3, 4)
top_words = word_freq.most_common(10)
if top_words:
    words, freqs = zip(*top_words)
    ax4.barh(words, freqs, color='teal')
    ax4.set_xlabel('빈도')
    ax4.invert_yaxis()
ax4.set_title('🔑 주요 키워드 TOP 10', fontsize=12, fontweight='bold')

# 5. 혐오 유형별 분포
ax5 = fig.add_subplot(2, 3, 5)
if type_freq:
    types, counts = zip(*type_freq.most_common())
    colors_hate = plt.cm.Reds(np.linspace(0.3, 0.9, len(types)))
    ax5.barh(types, counts, color=colors_hate)
    ax5.set_xlabel('빈도')
    ax5.invert_yaxis()
ax5.set_title('🚫 혐오표현 유형', fontsize=12, fontweight='bold')

# 6. 신뢰도 vs 좋아요
ax6 = fig.add_subplot(2, 3, 6)
scatter_colors = df_comments['has_hate'].map({True: '#e74c3c', False: '#3498db'})
ax6.scatter(df_comments['sentiment_confidence'], df_comments['likes'],
            c=scatter_colors, alpha=0.5, s=30)
ax6.set_xlabel('감성 신뢰도')
ax6.set_ylabel('좋아요')
ax6.set_title('📈 신뢰도 vs 좋아요 (빨강=혐오)', fontsize=12, fontweight='bold')

plt.suptitle(f'📊 "{KEYWORD}" YouTube API + KoBERT 분석 결과',
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# 7-2. 분석 결과 요약
# ============================================

print("\n" + "="*70)
print(f"📋 '{KEYWORD}' YouTube API + KoBERT 분석 결과")
print("="*70)

print(f"\n[1] 데이터 개요")
print(f"   - 검색 키워드: {KEYWORD}")
print(f"   - 분석 동영상: {df_comments['video_id'].nunique()}개")
print(f"   - 총 댓글 수: {len(df_comments):,}개")
print(f"   - 수집 방식: YouTube Data API v3")

print(f"\n[2] KoBERT 감성분석")
print(f"   - 모델: {MODEL_NAME if MODEL_LOADED else '규칙 기반'}")
for sentiment, count in df_comments['sentiment'].value_counts().items():
    pct = count / len(df_comments) * 100
    print(f"   - {sentiment}: {count}개 ({pct:.1f}%)")
print(f"   - 평균 신뢰도: {df_comments['sentiment_confidence'].mean():.3f}")

print(f"\n[3] 혐오표현 분석")
print(f"   - 혐오 포함: {df_comments['has_hate'].sum()}개 ({df_comments['has_hate'].mean()*100:.1f}%)")
if type_freq:
    top_hate = type_freq.most_common(3)
    print(f"   - 주요 유형: {', '.join([f'{t}({c})' for t, c in top_hate])}")

print(f"\n[4] 주요 키워드")
top_kw = word_freq.most_common(10)
print(f"   - {', '.join([w for w, c in top_kw])}")

print("\n" + "="*70)

In [ ]:
# ============================================
# 7-3. 결과 저장
# ============================================

output_cols = ['video_title', 'channel', 'text', 'cleaned_text', 'likes',
               'sentiment', 'sentiment_confidence', 'sentiment_score',
               'has_hate', 'hate_types', 'hate_words', 'hate_score', 'keywords']

df_output = df_comments[output_cols].copy()
df_output['keywords'] = df_output['keywords'].apply(lambda x: ', '.join(x) if x else '')
df_output['hate_types'] = df_output['hate_types'].apply(lambda x: ', '.join(x) if x else '')
df_output['hate_words'] = df_output['hate_words'].apply(lambda x: ', '.join(x) if x else '')

filename = f"youtube_api_kobert_analysis_{KEYWORD.replace(' ', '_')}.csv"
df_output.to_csv(filename, index=False, encoding='utf-8-sig')

print(f"✅ 결과가 '{filename}'로 저장되었습니다!")

# Colab에서 다운로드
try:
    from google.colab import files
    files.download(filename)
except:
    print("로컬 환경: 현재 디렉토리에서 파일을 확인하세요.")

---
# 🎓 학습 정리

## 이번 실습에서 배운 내용

### 1. YouTube Data API v3
- **공식 API**를 통한 안정적인 데이터 수집
- `google-api-python-client` 라이브러리 사용
- 동영상 검색, 댓글 수집 기능
- 일일 할당량: 10,000 units

### 2. API vs 크롤링 비교
| 항목 | YouTube API | 크롤링 |
|------|-------------|--------|
| 안정성 | ✅ 높음 | ⚠️ 변동 가능 |
| 속도 | ✅ 빠름 | 🐢 느림 |
| 제한 | 일일 할당량 | 차단 위험 |
| 데이터 품질 | ✅ 정확 | ⚠️ 파싱 오류 가능 |

### 3. KoBERT 감성분석
- 사전학습된 한국어 BERT 모델 활용
- Hugging Face transformers 라이브러리
- 신뢰도(confidence) 점수 제공

### 4. 혐오표현 탐지
- 7가지 유형의 패턴 기반 탐지
- 감성분석과 혐오탐지의 관계 분석

---

## 📚 참고 자료

- [YouTube Data API 문서](https://developers.google.com/youtube/v3)
- [Google Cloud Console](https://console.cloud.google.com/)
- [Hugging Face Korean Models](https://huggingface.co/models?language=ko)